## Level 4: Semantic Chunking <a id="SemanticChunking"></a>
Isn't it weird that we have a global constant for chunk size? Isn't it even weirder that our normal chunking mechanisms don't take into account the actual content?

I'm not the only one who thinks so

<!-- <div style="text-align: center;">
    <img src="static/SemanticChunkingtweet.png" style="max-width:50%; height:auto;"><br>
    <span><i><a href="https://twitter.com/thesephist/status/1724159343237456248?s=46">Source</a></i></span>
</div> -->

There has to be a better way - let's explore and find out.

Embeddings represent the semantic meaning of a string. They don't do much on their own, but when compared to embeddings of other texts you can start to infer the relationship between chunks. I want to lean into this property and explore using embeddings to find clusters of semantically similar texts.

The hypothesis is that semantically similar chunks should be held together.

I tried a few methods:
1) **Heirarchical clustering with positional reward** - I wanted to see how heirarchical clustering of sentence embeddings would do. But because I chose to split on sentences, there was an issue with small short sentences after a long one. You know? (like this last sentenence). They could change the meaning of a chunk, so I added a positional reward and clusters were more likely to form if they were sentences next to each other. This ended up being ok, but tuning the parameters was slow and unoptimal.
2) **Find break points between sequential sentences** - Next up I tried a walk method. I started at the first sentence, got the embedding, then compared it to sentence #2, then compared #2 and #3 and so on. I was looking for "break points" where embedding distance was large. If it was above a threshold, then I considered it the start of a new semantic section. I originally tried taking embeddings of every sentence, but this turned out to be too noisy. So I ended up taking groups of 3 sentences (a window), then got an embedding, then dropped the first sentence, and added the next one. This worked out a bit better.

I'll show method #2 here - It's not perfect by any means, but it's a good starting point for an exploration and I'd love to hear about how you think it could be improved.

First, let's load up our essay that we'll run through. I'm just doing a single essay here to keep the tokens down.

We'll be using Paul Graham's [MIT essay](https://paulgraham.com/mit.html)

Great, now that we have our sentences, I want to combine the sentence before and after so that we reduce noise and capture more of the relationships between sequential sentences.

Let's create a function so we can use it again. The `buffer_size` is configurable so you can select how big of a window you want. Keep this number in mind for the later steps. I'll just use `buffer_size=1` for now.

In [1]:
import spacy
import torch
import numpy as np
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\solom\miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('mit.txt') as file:
    document = file.read()

In [3]:
def get_sentences(document):
    return [str(sentence) for sentence in nlp(document).sents]

In [4]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [5]:
def get_token_length(sentences, tokenizer):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    return torch.sum(encoded_input["attention_mask"], axis=1)

In [6]:
def get_embeddings(sentences, model, tokenizer):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    respective_token_len = torch.sum(encoded_input["attention_mask"], axis=1)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # return sentence_embeddings, respective_token_len

    # Map into dictionary for ease of future lookup
    emb_token_pair = [{'embedding': emb, 'token_length': length} for emb, length in zip(sentence_embeddings, respective_token_len)]

    return dict(enumerate(emb_token_pair))

In [7]:
# a = (1,2,3)

In [8]:
# a + (5, 7)

In [9]:
# def testing(inputlist):
#     inputlist += (5, 7)

#     return inputlist

In [10]:
# import copy
# check = copy.deepcopy(a)

In [11]:
# testing(a)

In [12]:
# a

In [13]:
# def combine_sentences(indexed_sentences, buffer_size=1):
#     # Go through each sentence dict
#     for i in range(len(indexed_sentences)):

#         # Create a string that will hold the sentences which are joined
#         combined_sentence = ''

#         # Add sentences before the current one, based on the buffer size.
#         for j in range(i - buffer_size, i):
#             # Check if the index j is not negative (to avoid index out of range like on the first one)
#             if j >= 0:
#                 # Add the sentence at index j to the combined_sentence string
#                 combined_sentence += indexed_sentences[j]['sentence'] + ' '

#         # Add the current sentence
#         combined_sentence += indexed_sentences[i]['sentence']

#         # Add sentences after the current one, based on the buffer size
#         for j in range(i + 1, i + 1 + buffer_size):
#             # Check if the index j is within the range of the sentences list
#             if j < len(indexed_sentences):
#                 # Add the sentence at index j to the combined_sentence string
#                 combined_sentence += ' ' + indexed_sentences[j]['sentence']

#         # Then add the whole thing to your dict
#         # Store the combined sentence in the current sentence dict
#         indexed_sentences[i]['combined_sentence'] = combined_sentence
#         # sentences[i]['combined_sentences_indexes'] = [i-buffer_size, i, i+buffer_size] # Don't need the indexes anymore

#     return indexed_sentences

In [14]:
def combine_sentences(single_sentences_list, buffer_size=1):
    indexed_sentences = [{'sentence': x, 'index' : i} for i, x in enumerate(single_sentences_list)]
    # Go through each sentence dict
    for i in range(len(indexed_sentences)):

        # Create a string that will hold the sentences which are joined
        combined_sentence = ''

        # Add sentences before the current one, based on the buffer size.
        for j in range(i - buffer_size, i):
            # Check if the index j is not negative (to avoid index out of range like on the first one)
            if j >= 0:
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += indexed_sentences[j]['sentence'] + ' '

        # Add the current sentence
        combined_sentence += indexed_sentences[i]['sentence']

        # Add sentences after the current one, based on the buffer size
        for j in range(i + 1, i + 1 + buffer_size):
            # Check if the index j is within the range of the sentences list
            if j < len(indexed_sentences):
                # Add the sentence at index j to the combined_sentence string
                combined_sentence += ' ' + indexed_sentences[j]['sentence']

        # Then add the whole thing to your dict
        # Store the combined sentence in the current sentence dict
        indexed_sentences[i]['combined_sentence'] = combined_sentence
        # sentences[i]['combined_sentences_indexes'] = [i-buffer_size, i, i+buffer_size] # Don't need the indexes anymore

    return indexed_sentences

In [15]:
def combine_sentence_embeddings(combined_sentences, embeddings):
    for i, sentence in enumerate(combined_sentences):
        sentence['combined_sentence_embedding'] = embeddings[i]['embedding']
        sentence['token_length'] = embeddings[i]["token_length"]

    return combined_sentences

In [16]:
def calculate_cosine_distances(sentences):
    distances = []
    for i in range(len(sentences) - 1):
        embedding_current = sentences[i]['combined_sentence_embedding']
        embedding_next = sentences[i + 1]['combined_sentence_embedding']
        
        # Calculate cosine similarity
        similarity = cosine_similarity([embedding_current], [embedding_next])[0][0]
        
        # Convert to cosine distance
        distance = 1 - similarity

        # Append cosine distance to the list
        distances.append(distance)

        # Store distance in the dictionary
        sentences[i]['distance_to_next'] = distance

    # Optionally handle the last sentence
    # sentences[-1]['distance_to_next'] = None  # or a default value

    return distances, sentences

In [17]:
def calculate_outliners(distances, breakpoint_percentile_threshold):
    # We need to get the distance threshold that we'll consider an outlier
    # We'll use numpy .percentile() for this
    breakpoint_distance_threshold = np.percentile(distances, breakpoint_percentile_threshold) # If you want more chunks, lower the percentile cutoff
    
    # Then we'll see how many distances are actually above this one
    num_distances_above_theshold = len([x for x in distances if x > breakpoint_distance_threshold]) # The amount of distances above your threshold
    
    # Then we'll get the index of the distances that are above the threshold. This will tell us where we should split our text
    indices_above_thresh = [i for i, x in enumerate(distances) if x > breakpoint_distance_threshold] # The indices of those breakpoints on your list

    return indices_above_thresh

In [18]:
def get_combined_token_lengths(chunks):
    index_ranges = [range(i[0], i[1]) for i in [index[1] for index in chunks]]
    token_lengths = [sum([combined_sentences[each_index]['token_length'].item() for each_index in each_index_range]) for each_index_range in index_ranges]
    chunks_with_token_lengths = [(chunk[0], token_length) for chunk, token_length in zip(chunks, token_lengths)]
    
    return chunks_with_token_lengths

In [19]:
def get_chunk_lengths(chunks, combined_sentences):
    index_range_of_chunks = [range(index_pair[0], index_pair[1]) for index_pair in [chunk_index_pair[1] for chunk_index_pair in chunks]]
    token_lengths = [sum([combined_sentences[each_index]['token_length'].item() for each_index in each_index_range]) for each_index_range in index_range_of_chunks]
    chunk_lengths = [(chunk[0], token_length) for chunk, token_length in zip(chunks, token_lengths)]

    return chunk_lengths

In [20]:
def get_chunks(indices_above_thresh):
    # Initialize the start index
    start_index = 0
    
    # Create a list to hold the grouped sentences
    chunks = []
    
    # Iterate through the breakpoints to slice the sentences
    for index in indices_above_thresh:
        # The end index is the current breakpoint
        end_index = index
    
        # Slice the sentence_dicts from the current start index to the end index
        group = combined_sentences[start_index:end_index + 1]
        combined_text = ' '.join([d['sentence'] for d in group])
        chunks.append((combined_text, (start_index, end_index)))
        
        # Update the start index for the next group
        start_index = index + 1
    
    # The last group, if any sentences remain
    if start_index < len(combined_sentences):
        combined_text = ' '.join([d['sentence'] for d in combined_sentences[start_index:]])
        chunks.append((combined_text, (start_index, end_index)))

    return chunks

In [21]:
# Load in nlp model
nlp = spacy.load("en_core_web_sm")

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('D:\\DSAI\\Pre-Trained Models\\all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('D:\\DSAI\\Pre-Trained Models\\all-MiniLM-L6-v2')

C:\Users\solom\miniconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
single_sentences_list = get_sentences(document)

In [23]:
respective_sentence_length = get_token_length(single_sentences_list, tokenizer)

In [24]:
combined_sentences = combine_sentences(single_sentences_list)

In [25]:
embeddings = get_embeddings([x['combined_sentence'] for x in combined_sentences], model, tokenizer)

In [26]:
combined_sentences = combine_sentence_embeddings(combined_sentences, embeddings)

In [27]:
distances, combined_sentences = calculate_cosine_distances(combined_sentences)

In [28]:
# indices_above_thresh = calculate_outliners(distances, 95)

In [29]:
# chunks = get_chunks(indices_above_thresh)

In [30]:
# chunk_lengths = get_chunk_lengths(chunks, combined_sentences)

In [37]:
def get_optimal_chunks(distances, combined_sentences, token_limit=500, starting_threshold=95):
    indices_above_thresh = calculate_outliners(distances, starting_threshold)
    chunks = get_chunks(indices_above_thresh)
    chunk_lengths = get_chunk_lengths(chunks, combined_sentences)
    max_token_length = max([chunk_pair[1] for chunk_pair in chunk_lengths])
    print (starting_threshold, max_token_length)
    
    while max_token_length>token_limit:
        starting_threshold -= 5
        indices_above_thresh = calculate_outliners(distances, starting_threshold)
        chunks = get_chunks(indices_above_thresh)
        chunk_lengths = get_chunk_lengths(chunks, combined_sentences)
        max_token_length = max([chunk_pair[1] for chunk_pair in chunk_lengths])
        print (starting_threshold, max_token_length)

    return chunk_lengths
    # return [chunk_pairs[0] for chunk_pairs in chunk_lengths]

In [38]:
get_optimal_chunks(distances, combined_sentences)

95 4362
90 2942
85 1821
80 1198
75 1001
70 1001
65 1001
60 1001
55 1001
50 1001
45 1001
40 1001
35 341


[("Till recently graduating seniors had two choices: get a job or go to grad school. I think there will increasingly be a third option: to start your own startup. But how common will that be?\n\n I'm sure the default will always be to get a job, but starting a startup could well become as popular as grad school. In the late 90s my professor friends used to complain that they couldn't get grad students, because all the undergrads were going to work for startups.",
  203),
 ("I wouldn't be surprised if that situation returns, but with one difference: this time they'll be starting their own instead of going to work for other people's.\n\n The most ambitious students will at this point be asking: Why wait till you graduate? Why not start a startup while you're in college?",
  156),
 ("In fact, why go to college at all? Why not start a startup instead?\n\n A year and a half ago I gave a talk where I said that the average age of the founders of Yahoo, Google, and Microsoft was 24, and that i

In [33]:
# chunks

In [34]:
# chunk_lengths